# Text to audio using external API

## Text to audio using Multiple voices

In [5]:
from pydub import AudioSegment
import tempfile
import os
import tkinter as tk
from tkinter import filedialog
from genderize import Genderize
import pyttsx3
import spacy

# Load SpaCy model for named entity recognition
nlp = spacy.load("en_core_web_sm")

def get_gender_from_name(name):
    # Predict gender based on the name
    gender_info = Genderize().get([name])[0]
    return gender_info['gender']

def text_to_audio(name, text, gender='neutral', output_file=None):
    if output_file is None:
        output_file = f"{name}.mp3"

    engine = pyttsx3.init()

    if gender == 'male':
        voices = engine.getProperty('voices')
        engine.setProperty('voice', voices[0].id)  # Use the first voice (assumed to be male)
    elif gender == 'female':
        voices = engine.getProperty('voices')
        engine.setProperty('voice', voices[1].id)  # Use the second voice (assumed to be female)

    engine.save_to_file(text, output_file)
    engine.runAndWait()

def analyze_transcript(transcript):
    analyzed_lines = []
    for line in transcript.split('\n'):
        # Split the line into name and text
        parts = line.split(':')
        if len(parts) >= 2:
            name = parts[0].strip()
            text = parts[1].strip()
            # Predict gender based on the name
            gender = get_gender_from_name(name)
            analyzed_lines.append({'name': name, 'text': text, 'gender': gender})
    return analyzed_lines

def select_output_directory():
    root = tk.Tk()
    root.withdraw()  # Hide the main window

    # Ask the user to select the directory
    output_directory = filedialog.askdirectory(title="Select Output Directory")
    return output_directory

def combine_audio_files(audio_files, output_file):
    combined_audio = AudioSegment.empty()
    for file in audio_files:
        audio_segment = AudioSegment.from_file(file)
        combined_audio += audio_segment
    combined_audio.export(output_file, format="mp3")

# Multiple transcripts
transcripts = [
"""Abhishek : Hi Deepika, This is Abhishek from Facilities team, May I know the Problem?
Deepika :Hello Abhishek, there's a problem with Lack of cleaning services. Can you look into it?
Abhishek :Yes, of course. What exactly is the problem?
Deepika :It seems to be Lack of cleaning services in the workplace floor, and it's causing some inconvenience.
Abhishek :Understood, Can I know the Floor number and Exactly place?
Deepika : 4th floor and room number is near to podium.
Abhishek : We'll send someone to check it out and resolve the issue as quickly as possible and anything to assist.
Deepika : No Abhishek, Thank you!
Vikram : Thank you, I'm closing the conversation.""",
    
"""Rahul: Hi Priya, Rahul from Facilities here. What's the issue?
Priya: Hey Rahul, we're facing a problem with insufficient cleaning services. Can you please address it?
Rahul: Absolutely. Could you specify the issue?
Priya: It's a lack of cleaning on the 4th floor, near the podium, causing inconvenience.
Rahul: Got it. I'll send someone to check and resolve it quickly. Any other assistance needed?
Priya: No Rahul, thank you!
Amit: Thanks, closing the conversation.""",
 
"""John : Hi Sarah, How are you?
Sarah : I'm good, John. How about you?
John : I'm doing well, thanks for asking.
Sarah : Did you hear about the new project?
John : Yes, I did. It sounds exciting!
Sarah : I think so too. We have a lot of work ahead of us.
John : Absolutely, but I'm confident we can handle it.
Sarah : I agree. Let's schedule a meeting to discuss further details.
John : Sounds like a plan. Let's aim for tomorrow afternoon.
Sarah : Perfect. I'll send out the invites. Thanks, John!
John : No problem, Sarah. Have a great day!""",
    # Add more transcripts as needed
]

# Analyze and process each transcript
for idx, transcript in enumerate(transcripts):
    print(f"Processing Transcript {idx + 1}:")
    analyzed_transcript = analyze_transcript(transcript)
    
    # Generate audio files for each line in the transcript and save them in a temporary directory
    with tempfile.TemporaryDirectory() as temp_dir:
        audio_files = []
        for line in analyzed_transcript:
            output_file = os.path.join(temp_dir, f"{line['name'].replace(' ', '_')}_line{analyzed_transcript.index(line)}.mp3")
            text_to_audio(line['name'], line['text'], gender=line['gender'], output_file=output_file)
            audio_files.append(output_file)

        # Specify the output file path for the combined conversation
        output_directory = select_output_directory()
        if output_directory:
            combined_output_file = os.path.join(output_directory, f"combined_conversation_{idx + 1}.mp3")

            # Combine audio files into a single conversation
            combine_audio_files(audio_files, combined_output_file)

            print(f"Combined conversation {idx + 1} saved at:", combined_output_file)
        else:
            print("No output directory selected.")


Processing Transcript 1:
Combined conversation 1 saved at: C:/Users/gowtham.veepujerla/Downloads/Text to audio\combined_conversation_1.mp3
Processing Transcript 2:
Combined conversation 2 saved at: C:/Users/gowtham.veepujerla/Downloads/Text to audio\combined_conversation_2.mp3
Processing Transcript 3:
Combined conversation 3 saved at: C:/Users/gowtham.veepujerla/Downloads/Text to audio\combined_conversation_3.mp3


## Text to audio using local Folder

In [6]:
import os
import tkinter as tk
from tkinter import filedialog
import spacy
from genderize import Genderize
import pyttsx3
from IPython.display import Audio, display
import time


# Load SpaCy model for named entity recognition
nlp = spacy.load("en_core_web_sm")

def get_gender_from_name(name):
    # Predict gender based on the name
    gender_info = Genderize().get([name])[0]
    return gender_info['gender']

def text_to_audio(name, text, gender='neutral', output_file=None):
    # Initialize text-to-speech engine
    engine = pyttsx3.init()
    
    # Set voice based on gender
    if gender == 'male':
        voices = engine.getProperty('voices')
        engine.setProperty('voice', voices[0].id)  # Use the first voice (assumed to be male)
    elif gender == 'female':
        voices = engine.getProperty('voices')
        engine.setProperty('voice', voices[1].id)  # Use the second voice (assumed to be female)
    
    # Generate audio from text and save to file
    if output_file is None:
        with NamedTemporaryFile(delete=False, suffix=".mp3") as temp_audio:
            output_file = temp_audio.name
    engine.save_to_file(text, output_file)
    engine.runAndWait()

    # Display audio widget
    display(Audio(output_file, autoplay=True))

def analyze_transcript(transcript):
    analyzed_lines = []
    for line in transcript.split('\n'):
        # Split the line into name and text
        parts = line.split(':')
        if len(parts) >= 2:
            name = parts[0].strip()
            text = parts[1].strip()
            # Predict gender based on the name
            gender = get_gender_from_name(name)
            analyzed_lines.append({'name': name, 'text': text, 'gender': gender})
    return analyzed_lines

def select_output_directory():
    # Function to select output directory using Tkinter dialog
    root = tk.Tk()
    root.withdraw()
    output_directory = filedialog.askdirectory(title="Select Output Directory")
    return output_directory

def text_to_audio_batch(transcripts, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    audio_paths = []
    for idx, transcript in enumerate(transcripts):
        analyzed_transcript = analyze_transcript(transcript)
        for line_idx, line in enumerate(analyzed_transcript):
            output_file = os.path.join(output_folder, f"{line['name'].replace(' ', '_')}_{line_idx}.mp3")
            text_to_audio(line['name'], line['text'], gender=line['gender'], output_file=output_file)
            audio_paths.append(output_file)
    return audio_paths

# Example usage
transcripts = [
    """Abhishek : Hi Deepika, This is Abhishek from Facilities team, May I know the Problem?
Deepika :Hello Abhishek, there's a problem with Lack of cleaning services. Can you look into it?
Abhishek :Yes, of course. What exactly is the problem?
Deepika :It seems to be Lack of cleaning services in the workplace floor, and it's causing some inconvenience.
Abhishek :Understood, Can I know the Floor number and Exactly place?
Deepika : 4th floor and room number is near to podium.
Abhishek : We'll send someone to check it out and resolve the issue as quickly as possible and anything to assist.
Deepika : No Abhishek, Thank you!
Vikram : Thank you, I'm closing the conversation."""
]

output_folder = select_output_directory()
if output_folder:
    audio_paths = text_to_audio_batch(transcripts, output_folder)
    print("Audio files generated and saved at:")
    for path in audio_paths:
        print(path)


Audio files generated and saved at:
C:/Users/gowtham.veepujerla/Downloads\Abhishek_0.mp3
C:/Users/gowtham.veepujerla/Downloads\Deepika_1.mp3
C:/Users/gowtham.veepujerla/Downloads\Abhishek_2.mp3
C:/Users/gowtham.veepujerla/Downloads\Deepika_3.mp3
C:/Users/gowtham.veepujerla/Downloads\Abhishek_4.mp3
C:/Users/gowtham.veepujerla/Downloads\Deepika_5.mp3
C:/Users/gowtham.veepujerla/Downloads\Abhishek_6.mp3
C:/Users/gowtham.veepujerla/Downloads\Deepika_7.mp3
C:/Users/gowtham.veepujerla/Downloads\Vikram_8.mp3


In [9]:
from pydub import AudioSegment
import tkinter as tk
from tkinter import filedialog
import os

def select_input_files():
    root = tk.Tk()
    root.withdraw()
    input_files = filedialog.askopenfilenames(title="Select Audio Files")
    return input_files

def select_output_file():
    root = tk.Tk()
    root.withdraw()
    output_file = filedialog.asksaveasfilename(defaultextension=".mp3", filetypes=[("MP3 files", "*.mp3")], title="Save Combined Audio As")
    return output_file

def combine_audio_files(input_files, output_file):
    combined_audio = AudioSegment.empty()
    for file in input_files:
        audio_segment = AudioSegment.from_file(file)
        combined_audio += audio_segment
    combined_audio.export(output_file, format="mp3")

# Select input audio files
input_files = select_input_files()

if input_files:
    # Select output file path
    output_file = select_output_file()

    if output_file:
        # Combine audio files into a single conversation
        combine_audio_files(input_files, output_file)
        print("Combined conversation saved at:", output_file)
    else:
        print("Output file selection canceled.")
else:
    print("No input files selected.")


Combined conversation saved at: C:/Users/gowtham.veepujerla/Downloads/Project/combined_audio.mp3


In [19]:
import os
import time
from gtts import gTTS

# Function to generate time clip with male voice
def generate_time_clip(current_time, save_location):
    print("Generating time clip for:", current_time)
    text = f"It's {current_time}"
    print("Generated text:", text)
    file_path = os.path.join(save_location, f"time_clip_{current_time.replace(':', '_').replace(' ', '')}.mp3")
    print("File path:", file_path)
    if not os.path.exists(save_location):
        os.makedirs(save_location)
    try:
        tts = gTTS(text=text, lang='en', slow=False)
        tts.save(file_path)
        # Play the generated audio file
        os.startfile(file_path)
    except Exception as e:
        print("An error occurred:", e)

# Define the location to save the files
save_location = r"C:\Users\gowtham.veepujerla\Downloads\Text to audio\audio_files"

# Generate time announcements for each 15-minute interval
def generate_time_announcements(start_time, end_time, save_location):
    print("Generating time announcements...")
    current_time = start_time
    while current_time <= end_time:
        print("Current time:", current_time)
        generate_time_clip(current_time, save_location)
        # Update current time by adding 15 minutes
        current_time = (time.strptime(current_time, "%I:%M %p") + time.timedelta(minutes=15)).strftime("%I:%M %p")

# Set initial time to 12:00 AM (Indian clock format)
start_time = "12:00 AM"
# Set end time to 11:45 PM (Indian clock format)
end_time = "11:45 PM"

# Generate time announcements for each 15-minute interval
generate_time_announcements(start_time, end_time, save_location)


Generating time announcements...
